##MARKET BASKET ANALYSIS
######In the dynamic world of retail and e-commerce, understanding consumer behavior is key to success.
######In this analysis I am employing the powerful Apriori algorithm and Association Rule Mining techniques to unravel patterns hidden within transactional data. By examining the 'market basket' of items purchased together, we aim to reveal valuable insights into customer preferences, buying habits, and item relationships. Through this analysis, retailers can optimize product placement, cross-selling strategies, and promotional efforts, ultimately enhancing customer satisfaction and driving revenue growth.  


In [1]:
!pip install apyori

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install mlxtend

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install openpyxl


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
file_url = 'https://docs.google.com/spreadsheets/d/1nEhBA0wfim7djhOuiWC4OEwWpkUxaJXS/edit?usp=sharing&ouid=109906024218806745653&rtpof=true&sd=true'

csv_url = file_url.replace('/edit?usp=sharing', '/export?format=csv')

output = 'data.csv'
gdown.download(csv_url, output, quiet=False)

data = pd.read_csv(output)

print(data.head())


Downloading...
From: https://docs.google.com/spreadsheets/d/1nEhBA0wfim7djhOuiWC4OEwWpkUxaJXS/export?format=csv&ouid=109906024218806745653&rtpof=true&sd=true
To: /content/data.csv
39.6MB [00:02, 19.8MB/s]


   BillNo                             Itemname  Quantity          Date  Price  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/10 8:26   2.55   
1  536365                  WHITE METAL LANTERN         6  12/1/10 8:26   3.39   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  12/1/10 8:26   2.75   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/10 8:26   3.39   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/10 8:26   3.39   

   CustomerID         Country  
0     17850.0  United Kingdom  
1     17850.0  United Kingdom  
2     17850.0  United Kingdom  
3     17850.0  United Kingdom  
4     17850.0  United Kingdom  


In [ ]:
data.shape

(522064, 7)

In [ ]:
data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  float64
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [ ]:
data.describe()

,Quantity,Price,CustomerID
count,522064.000000,522064.000000,388023.000000
mean,10.090435,3.826801,15316.931710
std,161.110525,41.900599,1721.846964
min,-9600.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13950.000000
50%,3.000000,2.080000,15265.000000
75%,10.000000,4.130000,16837.000000
max,80995.000000,13541.330000,18287.000000


In [ ]:
print("Missing Values:")
print(data.isnull().sum())
data.dropna(inplace=True)

Missing Values:
BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64


In [ ]:
transaction_data = data.groupby(['BillNo', 'Date'])['Itemname'].apply(lambda x: ', '.join(x)).reset_index()
columns_to_drop = ['BillNo', 'Date']
transaction_data.drop(columns=columns_to_drop, inplace=True)

transaction_data_path = '/content/drive/MyDrive/Market Basket Analysis_MBA_ata.csv'
transaction_data.to_csv(transaction_data_path, index=False)

In [ ]:
print("\nTransaction Data for Association Rule Mining:")
print(transaction_data.head())
transaction_data.shape


Transaction Data for Association Rule Mining:
                                            Itemname
0  WHITE HANGING HEART T-LIGHT HOLDER, WHITE META...
1  HAND WARMER UNION JACK, HAND WARMER RED POLKA DOT
2  ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHOU...
3  JAM MAKING SET WITH JARS, RED COAT RACK PARIS ...
4                           BATH BUILDING BLOCK WORD


(18194, 1)

In [ ]:
items_df = transaction_data['Itemname'].str.split(', ', expand=True)
transaction_data = pd.concat([transaction_data, items_df], axis=1)
transaction_data = transaction_data.drop('Itemname', axis=1)
print(transaction_data.head())

                                  0                            1    \
0  WHITE HANGING HEART T-LIGHT HOLDER          WHITE METAL LANTERN   
1              HAND WARMER UNION JACK    HAND WARMER RED POLKA DOT   
2       ASSORTED COLOUR BIRD ORNAMENT    POPPY'S PLAYHOUSE BEDROOM   
3            JAM MAKING SET WITH JARS  RED COAT RACK PARIS FASHION   
4            BATH BUILDING BLOCK WORD                         None   

                              2                                    3    \
0  CREAM CUPID HEARTS COAT HANGER  KNITTED UNION FLAG HOT WATER BOTTLE   
1                            None                                 None   
2       POPPY'S PLAYHOUSE KITCHEN    FELTCRAFT PRINCESS CHARLOTTE DOLL   
3  YELLOW COAT RACK PARIS FASHION         BLUE COAT RACK PARIS FASHION   
4                            None                                 None   

                              4                                   5    \
0  RED WOOLLY HOTTIE WHITE HEART.        SET 7 BABUSHKA NESTI

In [ ]:
df_encoded = pd.get_dummies(transaction_data, prefix='', prefix_sep='').groupby(level=0, axis=1).max()
df_encoded.to_csv('transaction_data_encoded.csv', index=False)

In [ ]:
df_encoded = pd.read_csv('transaction_data_encoded.csv')

from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df_encoded, min_support=0.007, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

print("Association Rules:")
print(rules.head())

In [ ]:
import plotly.express as px


import pandas as pd
data = {
    'support': [0.1, 0.2, 0.3, 0.4, 0.5],
    'confidence': [0.6, 0.7, 0.8, 0.9, 1.0],
    'lift': [1.1, 1.2, 1.3, 1.4, 1.5]
}
rules = pd.DataFrame(data)

fig = px.scatter(
    rules,
    x="support",
    y="confidence",
    size="lift",
    color="lift",
    hover_name="lift",
    log_x=True,
    size_max=60,
    color_continuous_scale='Viridis',
    title="Market Basket Analysis - Support vs. Confidence (Size = Lift)"
)

fig.update_layout(
    xaxis_title="Support",
    yaxis_title="Confidence",
    legend_title_text="Lift",
    template="plotly_white"
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Example DataFrame (ensure your DataFrame is structured similarly)
# This is just a sample structure for illustration. Replace it with your actual DataFrame.
rules = pd.DataFrame({
    'antecedents': [frozenset({'milk'}), frozenset({'bread'}), frozenset({'butter'})],
    'consequents': [frozenset({'bread'}), frozenset({'butter'}), frozenset({'milk'})],
    'support': [0.2, 0.3, 0.1],
    'confidence': [0.6, 0.7, 0.5],
    'lift': [1.2, 1.3, 1.1]
})

# Convert frozensets to lists for serialization
if 'antecedents' in rules.columns:
    rules['antecedents'] = rules['antecedents'].apply(list)
else:
    raise KeyError('The DataFrame does not contain the column "antecedents".')

if 'consequents' in rules.columns:
    rules['consequents'] = rules['consequents'].apply(list)
else:
    raise KeyError('The DataFrame does not contain the column "consequents".')

# Create an interactive scatter plot using plotly express
fig = px.scatter(rules, x="support", y="confidence", size="lift",
                 color="lift", hover_name="consequents",
                 title='Market Basket Analysis - Support vs. Confidence',
                 labels={'support': 'Support', 'confidence': 'Confidence'})

# Customize the layout
fig.update_layout(
    xaxis_title='Support',
    yaxis_title='Confidence',
    coloraxis_colorbar_title='Lift',
    showlegend=True
)

# Show the interactive plot
fig.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go

G = nx.DiGraph()
for idx, row in rules.iterrows():
    G.add_node(tuple(row['antecedents']), color='skyblue')
    G.add_node(tuple(row['consequents']), color='orange')
    G.add_edge(tuple(row['antecedents']), tuple(row['consequents']), weight=row['support'])

pos = nx.spring_layout(G)
edge_x = []
edge_y = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        )
    )
)

layout = go.Layout(
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
)

fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
fig.show()

In [ ]:
import plotly.express as px
rules['rule'] = rules['antecedents'].astype(str) + ' -> ' + rules['consequents'].astype(str)

fig = px.sunburst(rules, path=['rule'], values='lift',
                  title='Market Basket Analysis - Sunburst Chart',
                  color='support', color_continuous_scale='YlGnBu')

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=40),
)
fig.show()